Source: http://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html#sphx-glr-intermediate-seq2seq-translation-tutorial-py

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

use_cuda = torch.cuda.is_available()

In [2]:
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    #put a space between punctuation, so not included in word
    s = re.sub(r"([.!?])", r" \1", s)
    #remove things that are not letters or punctuation
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def unicodeToAscii(s):
    # basically removes non-english standard alphabet letters and replaces with equivalent.
    # so no accented letters => less to learn
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def readLangs(input_file_location, lang1_name="English", lang2_name="French"):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open(input_file_location, encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    input_lang = Lang(lang1_name)
    output_lang = Lang(lang2_name)

    return input_lang, output_lang, pairs

In [3]:
# To make learning easier, we will limit to sentences less than max length 
# in either language
# And only take sentences that start with certain prefixes
# Removed punctuation b/c filtered out in our normalize function

MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[0].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


def prepareData(input_file_location, lang1_name="English", 
                lang2_name="French"):
    input_lang, output_lang, pairs = readLangs(input_file_location, lang1_name,
                                              lang2_name)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [4]:
#start of sentence
SOS_token = 0
# end of sentence
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.word2count = {}
        self.n_words = 2
        
    def addSentence(self, sentence):
        for word in sentence.split(" "):
            self.addWord(word)
    
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [5]:
input_lang, output_lang, pairs = prepareData("../../../data/text/fra-eng/fra.txt")
print(random.choice(pairs))

Reading lines...
Read 149861 sentence pairs
Trimmed to 11589 sentence pairs
Counting words...
Counted words:
English 3016
French 4634
['i m learning basque .', 'je suis en train d apprendre le basque .']
